In [42]:
import numpy as np
from numpy import inf as inf
from scipy.stats import binom
from heapq import heappush, heappop

In [8]:
binom.rvs(1, 0.2, size=100).reshape((10, 10))

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0]])

In [23]:
def connectGraph(graph):
    """
    Esta función se asegura que la matriz de adyacencia que entra por parámetro defina
    un grafo conexo, lo hace agragando una conexión aleatoria que llega a un nodo 
    identificado como no conexo. 
    @param graph: Matriz de adyacencia de un grafo conexo.
    """
    n = len(graph)
    for i in range(n):
        #Suma entradas de la i esima columna sin contar la diagonal
        incoming = np.sum(np.delete(graph[:, i], i)) 
        if incoming == 0:
            pos = np.random.randint(0, n-1)
            # Nos aseguramos de no poner la nueva conexión en la diagonal
            pos = pos if pos < i else pos + 1
            graph[pos, i] = 1



def generateRandomConnectedGraph(n, sparse: float = 0.2):
    """
    Esta función genera la matriz de adyacencia de un grafo dirigido, con pesos entre 1 y 50. 
    simplemente conexo y con N nodos.
    @param n: Número de nodos del grafo
    @param sparse: indica, en promedio, el porcentaje de vertices que llegaran y saldrán de un nodo. 
    El rango es entre 0 y 1.
    return: Matriz de adyacencia del grafo y la matriz de pesos
    """
    # Una grafo simplemente conexo tendrá, por cada columna, al menos un valor distinto de
    # cero en las entradas que no son la perteneciente a la diagonal. i.e, para que sea
    # conexo, por cada nodo, siempre debe haber un vertice que llega a dicho nodo

    # Inicialización de los vertices del grafo. 
    adyacencia = np.double(binom.rvs(1, sparse, size=n*n).reshape((n, n)))
    connectGraph(adyacencia)
    pesos = np.multiply(adyacencia, np.random.randint(1, 51, (n, n)))
    pesos[pesos == 0] = inf # Si no hay conexión a un nodo el peso es infinito

    return adyacencia, pesos


In [102]:
grafo, pesos = generateRandomConnectedGraph(10, sparse = 0.3)
print(pesos)

[[21. inf inf inf 28. inf inf inf inf inf]
 [32. inf inf inf inf inf inf 47. 43. inf]
 [37. inf inf inf inf inf 24. inf inf 31.]
 [inf inf inf inf 26. inf inf inf inf inf]
 [inf 28. inf 10.  1. 43. inf inf 42. inf]
 [34. inf inf 18. inf 42. inf inf inf inf]
 [inf inf inf 13. 22. inf inf 30. inf inf]
 [13. inf 36. inf inf inf inf inf inf inf]
 [inf inf inf  7. inf inf inf inf inf inf]
 [inf 40. inf inf 29.  2. inf 40. inf 38.]]


In [103]:
print(pesos)

[[21. inf inf inf 28. inf inf inf inf inf]
 [32. inf inf inf inf inf inf 47. 43. inf]
 [37. inf inf inf inf inf 24. inf inf 31.]
 [inf inf inf inf 26. inf inf inf inf inf]
 [inf 28. inf 10.  1. 43. inf inf 42. inf]
 [34. inf inf 18. inf 42. inf inf inf inf]
 [inf inf inf 13. 22. inf inf 30. inf inf]
 [13. inf 36. inf inf inf inf inf inf inf]
 [inf inf inf  7. inf inf inf inf inf inf]
 [inf 40. inf inf 29.  2. inf 40. inf 38.]]


In [106]:
def dijkstra(pesos, init):

    n = len(pesos)

    dist = np.ones(n) * inf
    visited = np.array([False for i in range(n)])
    path = np.ones(n) * -1

    dist[init] = 0
    visited[init] = True

    heap = []
    # Priority queue. Se ordena el heap basado el la primera entrada de la tupla 
    # (distancia, posicion) que se le pasa. Sacar el mínimo es O(1)
    heappush(heap, (dist[init], init))

    while len(heap) > 0:

        (minimo, nodoMin) = heappop(heap)
        nodoMin = int(nodoMin)
        visited[nodoMin] = True

        # Se recorren los nodos con pesos menores a infinito (los vecinos)
        for v in (v for v, peso in enumerate(pesos[nodoMin, :]) if peso < inf):
            if not visited[v]:
                # print('min: ',  nodoMin, 'v:', v, '-', dist[nodoMin], '-', pesos[nodoMin, v])

                # Si el nodo no ha sido visitado y la distancia desde el actual es menor
                # Entonces se actualiza su distancia
                if dist[v] > dist[nodoMin] + pesos[nodoMin, v]:

                    dist[v] = dist[nodoMin] + pesos[nodoMin, v]
                    path[v] = nodoMin
                    heappush(heap, (dist[v], v))

    return path


In [104]:
path = dijkstra(pesos, 3)

min:  3 v: 4 - 0.0 - 26.0
min:  4 v: 1 - 26.0 - 28.0
min:  4 v: 5 - 26.0 - 43.0
min:  4 v: 8 - 26.0 - 42.0
min:  1 v: 0 - 54.0 - 32.0
min:  1 v: 7 - 54.0 - 47.0
min:  1 v: 8 - 54.0 - 43.0
min:  5 v: 0 - 69.0 - 34.0
min:  7 v: 2 - 101.0 - 36.0
min:  2 v: 6 - 137.0 - 24.0
min:  2 v: 9 - 137.0 - 31.0


In [105]:
print(path)

[ 1.  4.  7. -1.  3.  4.  2.  1.  4.  2.]


In [28]:
pesos[3, :]

array([38., inf, inf, inf, inf,  9., inf, inf, inf, inf])

In [38]:
np.sum(np.array([1, 2, 3])[~np.array([True, True, False])])

3

In [44]:
h = []
heappush(h, (1, 3))
heappush(h, (0, 4))

print(heappop(h))

(0, 4)


In [66]:
for (i, n) in enumerate([1, 2, 3]):
    print(i, n)

0 1
1 2
2 3
